In [ ]:
# these networks are called DQN(deep Q- networks)

In [ ]:
# we are doing cart prblem discrete type

In [1]:
import random
import gym
import numpy as np

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

Using TensorFlow backend.


In [3]:
from collections import deque # used for experience-replay

In [ ]:
#agent is a player whicg takes action

In [5]:
class DQNAgent:
    
    def __init__(self, state_size, action_size):
        self.state_size=state_size
        self.action_size=action_size
        self.gamma=0.95     # discount factor
        self.epsilon=1.0   # we are not following q table initially we are taking random paths
        self.epsilon_decay=0.995  # at what rate we are decresung epsilon value so that it follows q table
        self.epsilon_min=0.01  # epsilon should not go beyond this value
        self.learning_rate=0.001
        self.memory=deque(maxlen=2000)
        self.model=self.build_model()  # build model is when called it return into model
        
    def build_model(self):
        model=Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        return model
        
    def act(self,state):
        if np.random.rand()<= self.epsilon:
            return random.randrange(self.action_size)
        act_values=self.model.predict(state)  # gives predicted q values
        return np.argmax(act_values[0]) # this returns index of max q value i.e index of action to be taken
        
    def remember(self, state,action, reward, next_state, done):
        self.memory.append((state,action, reward, next_state, done))
        
    def replay(self, batch_size):  # batch_size means kitne expreiences pe retrain karna hain
        minibatch=random.sample(self.memory, batch_size)
        for (state,action, reward, next_state, done) in minibatch:
            target=reward
            if not done:
                target += self.gamma* np.amax(self.model.predict(next_state)[0]) #max and amax are same max basically calls max
            target_matrix=self.model.predict(state)
            target_matrix[0][action]=target
            self.model.fit(state, target_matrix, epochs=1, verbose=0)
        if self.epsilon>self.epsilon_min:
            self.epsilon*=self.epsilon_decay

In [7]:
episodes= 50  #for 100 times can try

env=gym.make("CartPole-v1")
state_size=env.observation_space.shape[0]
action_size=env.action_space.n
agent=DQNAgent(state_size,action_size)
done=False  # starting done is false
batch_size= 32  #out of 2000 32 experiences are randomly chosen

for e in range(episodes):
    state=env.reset()
    state=np.reshape(state, [1, state_size])
    for time in range(500):
        #env.render()
        action=agent.act(state)
        next_state, reward, done, _ =env.step(action)
        reward=reward if not done else -10
        next_state=np.reshape(next_state, [1,state_size])
        agent.remember(state, action, reward, next_state,done)
        state=next_state
        if done:
            print(f"Episode:{e}/{episodes} Time:{time} Explore:{agent.epsilon}")
            break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
    #env.close()

 

Episode:0/50 Time:17 Explore:1.0
Episode:1/50 Time:10 Explore:1.0
Episode:2/50 Time:24 Explore:0.9000874278732445
Episode:3/50 Time:15 Explore:0.8348931673187264
Episode:4/50 Time:30 Explore:0.7183288830986236
Episode:5/50 Time:12 Explore:0.6763948591909945
Episode:6/50 Time:14 Explore:0.6305556603555866
Episode:7/50 Time:13 Explore:0.5907768628656763
Episode:8/50 Time:13 Explore:0.5535075230322891
Episode:9/50 Time:12 Explore:0.5211953074858876
Episode:10/50 Time:34 Explore:0.43952667968844233
Episode:11/50 Time:28 Explore:0.3819719776053028
Episode:12/50 Time:27 Explore:0.33362200135903064
Episode:13/50 Time:127 Explore:0.17651675623376062
Episode:14/50 Time:45 Explore:0.14087196468590776
Episode:15/50 Time:45 Explore:0.11242507996344034
Episode:16/50 Time:37 Explore:0.09339361644125409
Episode:17/50 Time:54 Explore:0.0712465030521374
Episode:18/50 Time:47 Explore:0.056292216338080694
Episode:19/50 Time:62 Explore:0.04125526245077418
Episode:20/50 Time:43 Explore:0.03325613897196732


In [ ]:
#Time tells ki kitne steps baad woh gir jayega

In [ ]:
#keras-rl use for reinforcement learning